# Spectral evaluation and modeling (fooof) of task MSL (ctr and sec)

Este script permite obtener los espectros de frecuencia (PSD) de cada una de las epocas de las tareas control y secuencia de MSL. TOmas las epocas de distintas formas, considerando solo el periodo de REST, solo el periodo de TASK o todo junto.

Guarda:
* Las PSD de cada epoca y cada canal.
* Los parametros del modelo FOOOF de cada epoca y cada canal e imagenes para analizar visualmente los espectros y modelado.
* Tablas .csv en el que se agrupan todos los parametros de los picos espectrales obtenidos con FOOOF

In [1]:
%matplotlib qt5

In [2]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import mne_bids 
import mne_bids.utils 
import pandas as pd
from fooof import FOOOF,FOOOFGroup
from fooof.analysis import get_band_peak_fm
import os

Funciones custom

In [3]:
def comp_psd(epochs, fname, time_range_labels_val, time_range_val, freq_range_val, bandwidth):
    '''
    This function takes an MNE epochsArray and computes the power spectral 
    density (PSD). Spectra are calculated for several specified time windows.

    Source: https://github.com/voytekresearch/oscillation_vs_exponent/blob/main/code/2_time_frequency_analysis.py
    '''
    
    # for the pre-stimulus and post-stimulus condition
    for label, time_range in zip(time_range_labels_val, time_range_val):
        print(time_range[0],time_range[1])
        # calculate PSD
        '''
        psd, freq = mne.time_frequency.psd_multitaper(epochs,
                                        fmin = freq_range_val[0], 
                                        fmax = freq_range_val[1],
                                        tmin = time_range[0], 
                                        tmax = time_range[1],
                                        bandwidth = bandwidth,
                                        verbose=False)
        '''
        '''
        spectrum = epochs.compute_psd(tmin = time_range[0], tmax = time_range[1],
                                      fmin = freq_range_val[0], fmax = freq_range_val[1],
                                      method='welch',n_per_seg=2000, n_fft=8000, average="mean", verbose=True)
        '''
        
        spectrum = epochs.compute_psd(tmin = time_range[0], tmax = time_range[1], 
                                    fmin = freq_range_val[0], fmax = freq_range_val[1], method='multitaper', 
                                    picks = epochs.info['ch_names'], #para incluir canales 'bads'
                                    bandwidth = bandwidth, verbose=True)
                
        spectrum.plot(exclude=[])
        
        psd, freq = spectrum.get_data(exclude=[], return_freqs=True)

        # save power results
        fname_out = str.replace(fname, '_epo.fif', '_%s_psd' %(label))
        fname_out = fname + '_epo-%s' %(label)
        print(fname_out)
        np.savez(fname_out, psd = psd, freq = freq)


def get_single_channel_epochs(epochs, channel):
    '''
    This function takes an MNE epochsArray and creates an new epochsArray 
    containing the data for single specified channel. 
    Rejected trials are removed from epochsArray.

    Source: https://github.com/voytekresearch/oscillation_vs_exponent/blob/main/code/2_time_frequency_analysis.py
    '''
    
    # get data for channel
    lfp = epochs.get_data(picks=channel)

    # check data contains at least 2 trials
    # note: some channels in dataset contain all NaN; only 1 trial causes error
    if np.isnan(lfp).all() or len(lfp) < 2:
        epochs_chan = 'no data'
        
    else:
        # remove missing/rejected data trials
        lfp = lfp[~np.isnan(lfp[:, 0, 0])]
    
        #create MNE epochs array
        info = mne.create_info(np.shape(lfp)[1], epochs.info['sfreq'], \
                               ch_types='eeg')
        epochs_chan = mne.EpochsArray(lfp, info, tmin=epochs.tmin, verbose=False)
    
    return epochs_chan

def compute_tfr(epochs, freq_range_val, method='multitaper', average_trials=False):
    '''
    This function takes an MNE epochsArray and computes the time-frequency
    representatoin of power using either multitapers or wavelets. 
    Time-frequnecy parameters are set to replicate Fellner et al.
    Due to memory demands, this function should be run on single-channel data, 
    or results can be averaged across trials.
    '''
    
    if method == 'multitaper':
        # set paramters for TF decomposition
        freq = np.logspace(*np.log10([freq_range_val[0],freq_range_val[1]]),128)
        n_cycles = freq / (1/0.3) # 300 ms - as published
        time_bandwidth = 10 * 0.3 # 10 Hz (when T=0.3 sec) - as published
        
        # TF decomposition using multitapers
        tfr = mne.time_frequency.tfr_multitaper(epochs, freqs=freq, n_cycles=n_cycles, 
                             time_bandwidth=time_bandwidth,
                             use_fft=True, return_itc=False, 
                             average=average_trials, verbose=False)

        tfr.average().plot([0])

    # define variables to return
    tfr = tfr.data

    return tfr, freq

Ubicación del archivo correspondiente sigueindo el formato BIDS

In [4]:
deriv_bids_root = '/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS'
mne_bids.print_dir_tree(deriv_bids_root, max_depth=4)

|Derivatives_Proyecto_LFA-ENYS_BIDS/
|--- README
|--- Referencias_guia_tener_en_cuenta.odt
|--- dataset_description.json
|--- participants.json
|--- participants.tsv
|--- sub-04/
|------ ses-day1/
|--------- sub-04_ses-day1_scans.tsv
|--------- PSD_epochs/
|------------ sub-04_ses-day1_desc-ctrTask_spectrumPSD_epo-epoch.npz
|------------ sub-04_ses-day1_desc-ctrTask_spectrumPSD_epo-rest.npz
|------------ sub-04_ses-day1_desc-ctrTask_spectrumPSD_epo-task.npz
|------------ sub-04_ses-day1_desc-secTask_spectrumPSD_epo-epoch.npz
|------------ sub-04_ses-day1_desc-secTask_spectrumPSD_epo-rest.npz
|------------ sub-04_ses-day1_desc-secTask_spectrumPSD_epo-task.npz
|--------- ieeg/
|------------ sub-04_ses-day1_selectedChannels.tsv
|------------ sub-04_ses-day1_task-msl_desc-baselineSegm1_channels.tsv
|------------ sub-04_ses-day1_task-msl_desc-baselineSegm1_events.tsv
|------------ sub-04_ses-day1_task-msl_desc-baselineSegm1_ieeg.edf
|------------ sub-04_ses-day1_task-msl_desc-baselineSegm1_

In [5]:
session = 'day1'
datatype = 'ieeg'
subjects = mne_bids.get_entity_vals(deriv_bids_root, 'subject')
tasks = mne_bids.get_entity_vals(deriv_bids_root, 'task')
suffix = 'ieeg'

# Indicar el codigo de descripcion del archivo 'desc'
desc_posible = ['ctrTask','secTask']

print(mne_bids.get_entity_vals(deriv_bids_root, 'subject'))
print(mne_bids.get_entity_vals(deriv_bids_root, 'task'))
print(mne_bids.get_entity_vals(deriv_bids_root, 'session'))

['04', '06', '07']
['msl']
['day1']


In [6]:
idx_sub = 2 # Indice del sujeto analizado
idx_task = 0 # Indice de la tarea
desc = desc_posible[1]  # registro a analizar

# Frecuencias para el filtrado notch
FREQS_NOTCH = [50, 100, 150, 200, 250]

TIME_RANGE = np.array([[-10.0, 10.0],    # full epoch
                       [-10.0, 0.0],    # REST
                       [0.0, 10.0]])    # TASK

TIME_RANGE_LABELS = np.array(['epoch',
                              'rest',
                              'task'])

# Multitaper
BANDWIDTH = 4 # multitaper bandwidth
FREQ_RANGE = [0.5, 45] 

# Flags
save_spectrum = 0
save_fooof_models = 0
plot_spectrum_models = 1

   
# Se identifica el archivo a levantar
deriv_bids_path = mne_bids.BIDSPath(root=deriv_bids_root, subject = subjects[idx_sub], task = tasks[idx_task], 
                                    session=session, suffix=suffix, datatype=datatype, description=desc)
print(deriv_bids_path.match(ignore_json=True))

# Se levanta el archivo en un objeto mne
raw_mne_deriv = mne_bids.read_raw_bids(bids_path=deriv_bids_path, verbose=False)
raw_mne_deriv.resample(sfreq=512)
raw_mne_deriv.info

# Se levantan los canales a utilizar en el registro correspondiente
select_chann_fname = str(deriv_bids_path.directory) + '/sub-' + subjects[idx_sub] + '_ses-' + session + '_selectedChannels' + '.tsv'
chann_select_aux = pd.read_csv(select_chann_fname, delimiter='\t')
chann_select = chann_select_aux['name'].to_list()
print(chann_select)

# Se seleccionan los canales correspondientes
raw_mne_deriv_select = raw_mne_deriv.copy().pick_channels(chann_select)
print(raw_mne_deriv_select.info)
print(raw_mne_deriv_select.info['ch_names'])

# Filtrado notch de la señal seleccionada
raw_mne_deriv_select_filt = raw_mne_deriv_select.load_data().copy().notch_filter(freqs=FREQS_NOTCH, 
                                                                                method='spectrum_fit', filter_length='10s',
                                                                                picks=raw_mne_deriv_select.info['ch_names'])

# Epocas
# A partir de los eventos/triggers enviados durante el registro se confeccionan epocas en el registro
# Las treas comienzan con un periodo de REST (precedido por un evento/trgger que lo indican) y continuan con un evento 
# de TASK (tambien precedido por un trigger/evento) que lo indica. 
# Cada periodo REST/TASK dura 10 segundos y se van danto alternativamente hasta completar 15 de cada uno.
#
# Para eso voy a tomar como punto centrol el trigger del periodo de task y tomar 10 segundos para atras y 10 seg para adelante.

events_id = {'inicio_msl_task': 16} 
events_from_annot, event_dict = mne.events_from_annotations(raw_mne_deriv, event_id=events_id)
epochs_mne_deriv_select_filt = mne.Epochs(raw_mne_deriv_select_filt, events_from_annot, tmin=TIME_RANGE[0,0], tmax=TIME_RANGE[0,1],
                                          event_id = event_dict, baseline = None)
print(epochs_mne_deriv_select_filt.info)
print(epochs_mne_deriv_select_filt.event_id)

# Computo de la PSD para cada canal tomando las epocas completas, el periodo de REST y el periodo de TASK por separado
spectr_dir = str(deriv_bids_path.root) + '/sub-' + subjects[idx_sub] + '/ses-' + session + '/PSD_epochs' 
if not os.path.exists(spectr_dir):
    os.makedirs(spectr_dir)

spectr_fname = spectr_dir + '/sub-' + subjects[idx_sub] + '_ses-' + session + '_desc-'+ desc +'_spectrumPSD'

comp_psd(epochs_mne_deriv_select_filt, spectr_fname, TIME_RANGE_LABELS, TIME_RANGE, FREQ_RANGE, BANDWIDTH)


[BIDSPath(
root: /home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS
datatype: ieeg
basename: sub-07_ses-day1_task-msl_desc-secTask_ieeg.edf)]


/tmp/ipykernel_26445/538224513.py:32: RuntimeWarning: Omitted 30 annotation(s) that were outside data range.
  raw_mne_deriv = mne_bids.read_raw_bids(bids_path=deriv_bids_path, verbose=False)
/tmp/ipykernel_26445/538224513.py:32: RuntimeWarning: Did not find any electrodes.tsv associated with sub-07_ses-day1_task-msl_desc-secTask.

The search_str was "/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/**/ieeg/sub-07_ses-day1*electrodes.tsv"
  raw_mne_deriv = mne_bids.read_raw_bids(bids_path=deriv_bids_path, verbose=False)
/tmp/ipykernel_26445/538224513.py:32: RuntimeWarning: Did not find any coordsystem.json associated with sub-07_ses-day1_task-msl_desc-secTask.

The search_str was "/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/**/ieeg/sub-07_ses-day1*coordsystem.json"
  raw_mne_deriv = mne_bids.read_raw_bids(bids_path=deriv_bids_path, verbose=False)


['HKAD1', 'HKAD2', 'HKAD3', 'HQUD1', 'HQUD2', 'HQUD3', 'HKAI1', 'HKAI2', 'HKAI3', 'HQUI1', 'HQUI2', 'HQUI3']
<Info | 10 non-empty values
 bads: []
 ch_names: HKAI1, HKAI2, HKAI3, HQUI1, HQUI2, HQUI3, HKAD1, HKAD2, HKAD3, ...
 chs: 12 sEEG
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 experimenter: mne_anonymize
 highpass: 0.0 Hz
 lowpass: 256.0 Hz
 meas_date: 2022-12-28 13:19:52 UTC
 nchan: 12
 projs: []
 sfreq: 512.0 Hz
 subject_info: 3 items (dict)
>
['HKAI1', 'HKAI2', 'HKAI3', 'HQUI1', 'HQUI2', 'HQUI3', 'HKAD1', 'HKAD2', 'HKAD3', 'HQUD1', 'HQUD2', 'HQUD3']
Removed notch frequencies (Hz):
     50.00 :  708 windows
    100.00 :  708 windows
    150.00 :  708 windows
    200.00 :  708 windows
    249.00 :  708 windows
    250.00 :  708 windows
    251.00 :  708 windows
Used Annotations descriptions: ['inicio_msl_task']
Not setting metadata
15 matching events found
No baseline correction applied
0 projection items activated
<Info 

/tmp/ipykernel_26445/174072611.py:33: RuntimeWarning: Channel locations not available. Disabling spatial colors.
  spectrum.plot(exclude=[])


    Using multitaper spectrum estimation with 38 DPSS windows
Averaging across epochs...
/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/PSD_epochs/sub-07_ses-day1_desc-secTask_spectrumPSD_epo-rest
0.0 10.0
Using data from preloaded Raw for 15 events and 10241 original time points ...
    Using multitaper spectrum estimation with 38 DPSS windows


/tmp/ipykernel_26445/174072611.py:33: RuntimeWarning: Channel locations not available. Disabling spatial colors.
  spectrum.plot(exclude=[])


Averaging across epochs...
/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/PSD_epochs/sub-07_ses-day1_desc-secTask_spectrumPSD_epo-task


/tmp/ipykernel_26445/174072611.py:33: RuntimeWarning: Channel locations not available. Disabling spatial colors.
  spectrum.plot(exclude=[])


## Computo de los parametros con el modelo 'fooof' 

Link: https://fooof-tools.github.io/fooof/index.html

In [7]:
# 'fooof' SpecParam hyperparameters 
N_JOBS = -1 # number of jobs for parallel processing
SPEC_PARAM_SETTINGS = {
    'peak_width_limits' :   [2, 20], # default: (0.5, 12.0)) - recommends at least frequency resolution * 2
    'min_peak_height'   :   0.1, 
    'max_n_peaks'       :   6, # (default: inf)
    'peak_threshold'    :   2.0} # (default: 2.0)
aperiodic_mode = 'fixed' 

save_fooof_models = 1

for label in TIME_RANGE_LABELS: # np.array(['task']): #
    
    fname_psd = str.replace(spectr_fname, '_epo.fif', '_%s_psd' %(label))
    fname_psd = spectr_fname + '_epo-%s' %(label)
    print(fname_psd + '.npz')

    data_psd = np.load(fname_psd + '.npz')
    print(data_psd.files)
    #print(data_psd['psd'].shape)
    #print(type(data_psd['psd']))
    #print(data_psd['freq'].shape)

    for n_epoch in range(data_psd['psd'].shape[0]):
        print(n_epoch)
        # Modelado de los espectros utilizando la herramienta FOOOF
        fg_select = FOOOFGroup(**SPEC_PARAM_SETTINGS, aperiodic_mode = aperiodic_mode)
        fg_select.fit(data_psd['freq'], data_psd['psd'][n_epoch,:,:])

        # Para guardar el modelado
        if save_fooof_models == 1:

            spectr_fit_dir = str(deriv_bids_path.root) + '/sub-' + subjects[idx_sub] + '/ses-' + session + '/spectral_fit' + '/fooof_fit_params' 
            if not os.path.exists(spectr_fit_dir):
                os.makedirs(spectr_fit_dir)

            spectr_fit_fname = spectr_fit_dir + '/sub-' + subjects[idx_sub] + '_ses-' + session + '_desc-'+ desc +'_spectralFit' + '_epo-%s' %(label) + '_n-%s' %(n_epoch)
            print(spectr_fit_fname)
            fg_select.save(spectr_fit_fname, save_settings=True, save_results=True)

        
        # Para guardar figuras de control, las guardo para verlas posteriormente y que no se abran todas juntas

        sqrt = np.sqrt(len(fg_select.group_results)) # variable auxiliar para determinar la estructura del subplot

        # Busco los indices en 'y' y 'x' para graficar cada espectro en una misma figura
        if len(fg_select.group_results) == 3:
            max_idx_y = 1
            max_idx_x = 3
        elif sqrt%int(sqrt) == 0:
            max_idx_y = int(sqrt)
            max_idx_x = max_idx_y 
        else: 
            max_idx_y = int(np.round(sqrt))
            max_idx_x = int(np.ceil(sqrt))

        # Figuras
        fig1, (ax1) = plt.subplots(max_idx_y,max_idx_x, sharex=True,sharey=True) #
        fig2, (ax2) = plt.subplots(max_idx_y,max_idx_x, sharex=True,sharey=True) #

        # Variables auxiliares. OJO: ver que en realidad sean utiles y no hagan lio
        freqs = data_psd['freq'] 
        psds = data_psd['psd']

        for idx_y in range(max_idx_y):
            for idx_x in range(max_idx_x):
                chan_n = idx_y*max_idx_x + idx_x
                #print("chan:",chan_n)
                #print("y:",idx_y)
                #print("x:",idx_x)
                
                # Se obtiene el ajuste de cada uno de los canales
                fm = fg_select.get_fooof(chan_n, regenerate=True)
                
                aperiodic = fm.get_params('aperiodic_params', 'offset') - np.log10(freqs**fm.get_params('aperiodic_params', 'exponent'))
                
                
                if max_idx_y != 1:

                    fm.plot(ax=ax1[idx_y,idx_x],add_legend=False) 
                    
                    ax1[idx_y,idx_x].plot(freqs,np.log10(psds[n_epoch,chan_n,:]),'k')
                    ax1[idx_y,idx_x].set_title(epochs_mne_deriv_select_filt.info['ch_names'][chan_n])
                    
                    # Espectro suprimiendo la componente 1/f
                    ax2[idx_y,idx_x].plot(freqs,np.log10(psds[n_epoch,chan_n,:])-aperiodic,'k')
                    ax2[idx_y,idx_x].plot(fm.freqs,fm.fooofed_spectrum_-aperiodic,'r')
                    ax2[idx_y,idx_x].set_title(epochs_mne_deriv_select_filt.info['ch_names'][chan_n])
                    ax2[idx_y,idx_x].axhline(y=0, xmin=np.min(freqs), xmax=np.max(freqs))
                
                elif max_idx_y == 1:
                    
                    fm.plot(ax=ax1[idx_x],add_legend=False) 
                    
                    ax1[idx_x].plot(freqs,np.log10(psds[n_epoch,chan_n,:]),'k')
                    ax1[idx_x].set_title(epochs_mne_deriv_select_filt.info['ch_names'][chan_n])
                    
                    # Espectro suprimiendo la componente 1/f
                    ax2[idx_x].plot(freqs,np.log10(psds[n_epoch,chan_n,:])-aperiodic,'k')
                    ax2[idx_x].plot(fm.freqs,fm.fooofed_spectrum_-aperiodic,'r')
                    ax2[idx_x].set_title(epochs_mne_deriv_select_filt.info['ch_names'][chan_n])
                    ax2[idx_x].axhline(y=0, xmin=np.min(freqs), xmax=np.max(freqs))


        #Para guardar figuras por cada tipo de Epoca y # de epoca
        figure_fit_dir = str(deriv_bids_path.root) + '/sub-' + subjects[idx_sub] + '/ses-' + session + '/spectral_fit' + '/fooof_fit_figures' 
        if not os.path.exists(figure_fit_dir):
            os.makedirs(figure_fit_dir)

        figure_fit_fname = figure_fit_dir + '/sub-' + subjects[idx_sub] + '_ses-' + session + '_desc-'+ desc +'_spectralFit' + '_epo-%s' %(label) + '_n-%s' %(n_epoch) 
        print(figure_fit_fname + '_fooofed_spectrum')
        fig1.savefig(figure_fit_fname + '_fooofed_spectrum', dpi='figure', format='png')
        fig2.savefig(figure_fit_fname + '_flattened_spectrum', dpi='figure', format='png')

/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/PSD_epochs/sub-07_ses-day1_desc-secTask_spectrumPSD_epo-epoch.npz
['psd', 'freq']
0
Running FOOOFGroup across 12 power spectra.
/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/spectral_fit/fooof_fit_params/sub-07_ses-day1_desc-secTask_spectralFit_epo-epoch_n-0
/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/spectral_fit/fooof_fit_figures/sub-07_ses-day1_desc-secTask_spectralFit_epo-epoch_n-0_fooofed_spectrum
1
Running FOOOFGroup across 12 power spectra.
/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/spectral_fit/fooof_fit_params/sub-07_ses-day1_desc-secTask_spectralFit_epo-epoch_n-1
/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/spectral_fit/fooof_fit_figures/sub-07_ses-day1_desc-secTask_spectralFit_epo-epoch_n-1_fooofed_spectrum
2
Running FOOOFGroup across 12 power spectra.
/home/lfa-01/Documentos/Deriv

/tmp/ipykernel_26445/4150176175.py:59: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig2, (ax2) = plt.subplots(max_idx_y,max_idx_x, sharex=True,sharey=True) #


/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/spectral_fit/fooof_fit_figures/sub-07_ses-day1_desc-secTask_spectralFit_epo-epoch_n-8_fooofed_spectrum
9
Running FOOOFGroup across 12 power spectra.
/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/spectral_fit/fooof_fit_params/sub-07_ses-day1_desc-secTask_spectralFit_epo-epoch_n-9
/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/spectral_fit/fooof_fit_figures/sub-07_ses-day1_desc-secTask_spectralFit_epo-epoch_n-9_fooofed_spectrum
10
Running FOOOFGroup across 12 power spectra.
/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/spectral_fit/fooof_fit_params/sub-07_ses-day1_desc-secTask_spectralFit_epo-epoch_n-10
/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/spectral_fit/fooof_fit_figures/sub-07_ses-day1_desc-secTask_spectralFit_epo-epoch_n-10_fooofed_spectrum
11
Running FOOOFGroup across 12 power spectra.
/hom

A continuacion se levantan los parametros obtenidos con 'fooof' para crear una tabla con todo los parametros 
(incluyendo todos los canales y epocas, para todos loos tiempos indicados)

In [8]:
#Se recorren los datos de cada segmento y cada epoca
for label in TIME_RANGE_LABELS: # np.array(['task']): #
    
    fname_psd = str.replace(spectr_fname, '_epo.fif', '_%s_psd' %(label))
    fname_psd = spectr_fname + '_epo-%s' %(label)
    #print(fname_psd + '.npz')

    data_psd = np.load(fname_psd + '.npz')
    #print(data_psd.files)

    for n_epoch in range(data_psd['psd'].shape[0]):
        
        #Se lavantan los datos del ajuste fooof para crear tablas conjuntas
        spectr_fit_dir = str(deriv_bids_path.root) + '/sub-' + subjects[idx_sub] + '/ses-' + session + '/spectral_fit' + '/fooof_fit_params' 
        spectr_fit_fname = spectr_fit_dir + '/sub-' + subjects[idx_sub] + '_ses-' + session + '_desc-'+ desc +'_spectralFit' + '_epo-%s' %(label) + '_n-%s' %(n_epoch)
  
        fg_select = FOOOFGroup()
        fg_select.load(spectr_fit_fname)
        
        # Peak parameters are labeled as:
        # CF: center frequency of the extracted peak
        # PW: power of the peak, over and above the aperiodic component
        # BW: bandwidth of the extracted peak
        group_peak_params = fg_select.get_params('peak_params') 
        group_peak_params = np.column_stack((group_peak_params, np.ones(group_peak_params.shape[0])*n_epoch))
        #print(group_peak_params)

        if n_epoch == 0: # si es la primer epoca guardo en una tabla aparte para despues ir concatenando
            group_peak_params_allEpochs = group_peak_params
        else:
            group_peak_params_allEpochs = np.row_stack((group_peak_params_allEpochs, group_peak_params))
        
    #print(group_peak_params_allEpochs)

    # Nombres de las columnas
    col_names = ['CF','PW','BW','n_chan','n_epoch']
    df_group_peak_params_allEpochs = pd.DataFrame(group_peak_params_allEpochs, columns=col_names)

    # Gudardar DataFrame como .csv
    df_fit_params_fname = str(deriv_bids_path.root) + '/sub-' + subjects[idx_sub] + '/ses-' + session + '/spectral_fit' + '/sub-' + subjects[idx_sub] + '_ses-' + session + '_desc-'+ desc +'_spectralFit' + '_epo-%s' %(label) + '_all_epochs'
    print(df_fit_params_fname+'.csv')
    df_group_peak_params_allEpochs.to_csv(df_fit_params_fname + '.csv', index=True, header=True, sep=',')
    del group_peak_params_allEpochs, df_group_peak_params_allEpochs

'''
group_peak_params = fg.get_params('peak_params') 
print(np.ones(group_peak_params.shape[0]))
group_peak_params = np.column_stack((group_peak_params, np.ones(group_peak_params.shape[0])))

#group_peak_params[:,4] = 1
print(group_peak_params)
#print(fg.get_params('peak_params').shape)

fig, ax = plt.subplots()
ax.scatter(group_peak_params[:,4],group_peak_params[:,0], c=group_peak_params[:,3])

plt.show()
'''

/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/spectral_fit/sub-07_ses-day1_desc-secTask_spectralFit_epo-epoch_all_epochs.csv
/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/spectral_fit/sub-07_ses-day1_desc-secTask_spectralFit_epo-rest_all_epochs.csv
/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/spectral_fit/sub-07_ses-day1_desc-secTask_spectralFit_epo-task_all_epochs.csv


/home/lfa-01/anaconda3/envs/test_neo/lib/python3.10/site-packages/fooof/objs/group.py:378: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  out = np.array([np.insert(getattr(data, name), 3, index, axis=1)
/home/lfa-01/anaconda3/envs/test_neo/lib/python3.10/site-packages/fooof/objs/group.py:378: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  out = np.array([np.insert(getattr(data, name), 3, index, axis=1)
/home/lfa-01/anaconda3/envs/test_neo/lib/python3.10/site-packages/fooof/objs/group.py:378: VisibleDeprecationWarning: Creating an ndarray from ragged ne

"\ngroup_peak_params = fg.get_params('peak_params') \nprint(np.ones(group_peak_params.shape[0]))\ngroup_peak_params = np.column_stack((group_peak_params, np.ones(group_peak_params.shape[0])))\n\n#group_peak_params[:,4] = 1\nprint(group_peak_params)\n#print(fg.get_params('peak_params').shape)\n\nfig, ax = plt.subplots()\nax.scatter(group_peak_params[:,4],group_peak_params[:,0], c=group_peak_params[:,3])\n\nplt.show()\n"

### Graficar el promedio de los PSD
Promedio de los PSD entre epocas, para cada segmento

In [9]:
plt.close('all')

In [9]:
# PATH a las PSD para cada canal tomando las epocas completas, el periodo de REST y el periodo de TASK por separado
spectr_fname = str(deriv_bids_path.root) + '/sub-' + subjects[idx_sub] + '/ses-' + session + '/PSD_epochs' + '/sub-' + subjects[idx_sub] + '_ses-' + session + '_desc-'+ desc +'_spectrumPSD'

colors_arr = ['k','r','b']

for label,idx in zip(np.array(['task','rest']), range(2)): #TIME_RANGE_LABELS: # 
    
    fname_psd = str.replace(spectr_fname, '_epo.fif', '_%s_psd' %(label))
    fname_psd = spectr_fname + '_epo-%s' %(label)
    print(fname_psd + '.npz')

    data_psd = np.load(fname_psd + '.npz')

    # Media de los espectros a lo largo de las epocas
    mean_data_psd = np.mean(np.log10(data_psd['psd']), axis=0)
    std_data_psd = np.std(np.log10(data_psd['psd']), axis=0)

    if idx == 0:

        # Plot mean psd
        sqrt = np.sqrt(mean_data_psd.shape[0]) # variable auxiliar para determinar la estructura del subplot

        # Busco los indices en 'y' y 'x' para graficar cada espectro en una misma figura
        if mean_data_psd.shape[0] == 3:
            max_idx_y = 1
            max_idx_x = 3
        elif sqrt%int(sqrt) == 0:
            max_idx_y = int(sqrt)
            max_idx_x = max_idx_y 
        else: 
            max_idx_y = int(np.round(sqrt))
            max_idx_x = int(np.ceil(sqrt))

        # Figuras
        fig3, (ax3) = plt.subplots(max_idx_y,max_idx_x, sharex=True,sharey=True) #
    
    # Variables auxiliares
    freqs = data_psd['freq'] 
    mean_psds = mean_data_psd
    std_psds = std_data_psd 

    for idx_y in range(max_idx_y):
        for idx_x in range(max_idx_x):
            chan_n = idx_y*max_idx_x + idx_x
            #print(chan_n)
            #print("y:",idx_y)
            #print("x:",idx_x)

            if max_idx_y != 1:
                ax3[idx_y,idx_x].plot(freqs, mean_psds[chan_n,:],colors_arr[idx])
                ax3[idx_y,idx_x].fill_between(freqs, mean_psds[chan_n,:]-std_psds[chan_n,:], mean_psds[chan_n,:]+std_psds[chan_n,:], alpha=0.3, facecolor=colors_arr[idx])
                ax3[idx_y,idx_x].set_title(epochs_mne_deriv_select_filt.info['ch_names'][chan_n])
                ax3[idx_y,idx_x].set_xlabel('Freq [Hz]')
                ax3[idx_y,idx_x].set_ylabel('Power')
            elif max_idx_y == 1:
                ax3[idx_x].plot(freqs, mean_psds[chan_n,:],colors_arr[idx])
                ax3[idx_x].fill_between(freqs, mean_psds[chan_n,:]-std_psds[chan_n,:], mean_psds[chan_n,:]+std_psds[chan_n,:], alpha=0.3, facecolor=colors_arr[idx])
                ax3[idx_x].set_title(epochs_mne_deriv_select_filt.info['ch_names'][chan_n])
                ax3[idx_x].set_xlabel('Freq [Hz]')
                ax3[idx_x].set_ylabel('Power')

            #fig3.suptitle('PSG del segmento:' + label)
            fig3.show()
    
    

/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/PSD_epochs/sub-07_ses-day1_desc-secTask_spectrumPSD_epo-task.npz
/home/lfa-01/Documentos/Derivatives_Proyecto_LFA-ENYS_BIDS/sub-07/ses-day1/PSD_epochs/sub-07_ses-day1_desc-secTask_spectrumPSD_epo-rest.npz


Para graficar y guardar figuras de los fits con fooof

In [ ]:
sqrt = np.sqrt(len(fg_select.group_results))

# Busco los indices en 'y' y 'x' para graficar cada espectro en una misma figura
if sqrt%int(sqrt) == 0:
    max_idx_y = int(sqrt)
    max_idx_x = max_idx_y 
else: 
    max_idx_y = int(np.round(sqrt))
    max_idx_x = int(np.ceil(sqrt))

# Figura
fig1, (ax1) = plt.subplots(max_idx_y,max_idx_x, sharex=True,sharey=True) #
fig2, (ax2) = plt.subplots(max_idx_y,max_idx_x, sharex=True,sharey=True) #

freqs = data_psd['freq'] 
psds = data_psd['psd']

for idx_y in range(max_idx_y):
    for idx_x in range(max_idx_x):
        chan_n = idx_y*max_idx_y + idx_x
        print(chan_n)
        print("y:",idx_y)
        print("x:",idx_x)
        
        
        fm = fg_select.get_fooof(chan_n, regenerate=True)
        
        aperiodic = fm.get_params('aperiodic_params', 'offset') - np.log10(freqs**fm.get_params('aperiodic_params', 'exponent'))
        fm.plot(ax=ax1[idx_y,idx_x],add_legend=False) 
        
        
        ax1[idx_y,idx_x].plot(freqs,np.log10(psds[0,chan_n,:]),'k')
        ax1[idx_y,idx_x].set_title(epochs_mne_deriv_select_filt.info['ch_names'][chan_n])
        
        # Espectro suprimiendo la componente 1/f
        ax2[idx_y,idx_x].plot(freqs,np.log10(psds[0,chan_n,:])-aperiodic,'k')
        ax2[idx_y,idx_x].plot(fm.freqs,fm.fooofed_spectrum_-aperiodic,'r')
        ax2[idx_y,idx_x].set_title(epochs_mne_deriv_select_filt.info['ch_names'][chan_n])
        ax2[idx_y,idx_x].axhline(y=0, xmin=np.min(freqs), xmax=np.max(freqs))


#Para guardar figuras por cada tipo de Epoca y # de epoca
figure_fit_dir = str(deriv_bids_path.root) + '/sub-' + subjects[idx_sub] + '/ses-' + session + '/spectral_fit' + '/fooof_fit_figures' 
if not os.path.exists(figure_fit_dir):
    os.makedirs(figure_fit_dir)

figure_fit_fname = figure_fit_dir + '/sub-' + subjects[idx_sub] + '_ses-' + session + '_desc-'+ desc +'_spectralFit' + '_epo-%s' %(label) + '_n-%s' %(n_epoch) 
print(figure_fit_fname + '_fooofed_spectrum')
fig1.savefig(figure_fit_fname + '_fooofed_spectrum', dpi='figure', format='png')
fig2.savefig(figure_fit_fname + '_flattened_spectrum', dpi='figure', format='png')
        

plt.show()

### Para probar R

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
#mtcars
library(ggplot2)
#ggplot(mtcars, aes(x=wt, y=mpg)) + geom_point()

In [77]:
group_peak_params = fg_select.get_params('peak_params') 
print(group_peak_params)

[[3.09482508e+00 8.30576259e-01 1.93654589e+00 0.00000000e+00]
 [5.66521026e+00 6.95186851e-01 3.20249812e+00 0.00000000e+00]
 [1.13043707e+01 2.94781433e-01 6.48961227e+00 0.00000000e+00]
 [1.69105672e+01 3.38984073e-01 3.69995413e+00 0.00000000e+00]
 [4.28295968e+01 1.44452628e-01 3.08097315e+00 0.00000000e+00]
 [4.59019482e+01 1.43226619e-01 1.09920027e+00 0.00000000e+00]
 [6.00672832e+01 1.22615121e-01 5.52342066e+00 0.00000000e+00]
 [7.12700979e+01 1.60398880e-01 3.75489199e+00 0.00000000e+00]
 [7.64017866e+01 1.67871308e-01 3.16520416e+00 0.00000000e+00]
 [8.61334444e+01 1.13634273e-01 5.66815201e+00 0.00000000e+00]
 [9.70548142e+01 2.05365427e-01 6.46970017e+00 0.00000000e+00]
 [3.35214265e+00 7.24960384e-01 1.83936071e+00 1.00000000e+00]
 [5.99851290e+00 5.82487478e-01 2.29632424e+00 1.00000000e+00]
 [8.23446767e+00 3.28487485e-01 1.18275179e+00 1.00000000e+00]
 [9.72144624e+00 1.94367504e-01 1.54261618e+00 1.00000000e+00]
 [1.59525468e+01 2.41274159e-01 6.79138249e+00 1.000000

/home/lfa-01/anaconda3/envs/test_neo/lib/python3.10/site-packages/fooof/objs/group.py:378: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  out = np.array([np.insert(getattr(data, name), 3, index, axis=1)


In [7]:
#def compute_channel_tfr(epochs, fname, dir_output):
'''
This function takes an MNE epochsArray and computes the time-frequency
representatoin of power for each channel sequentially, saving the results
for each channel seperately. 
'''

TFR_METHOD = ['multitaper']
epochs = epochs_mne_deriv_select_filt
channel = 0

# get single channel epochs, and compute TFR for each channel
#for channel in range(len(epochs.info['ch_names'])):
    # get single channle epochs
     
epochs_chan = get_single_channel_epochs(epochs, channel)
    

# run time-frequency analysis
for method in TFR_METHOD:
    tfr, freq = compute_tfr(epochs_chan, FREQ_RANGE, method=method)
    
    # save time-frequency results
    #fname_out = str.replace(fname, '_epo.fif', \
    #                        '_chan%s_tfr_%s' %(channel, method))
    #np.savez(join(dir_output, fname_out), 
    #            tfr=tfr, freq=freq, time=epochs_chan.times)

Using data from preloaded Raw for 15 events and 10241 original time points ...
No baseline correction applied


Traceback (most recent call last):
  File "/home/lfa-01/anaconda3/envs/test_neo/lib/python3.10/site-packages/matplotlib/cbook/__init__.py", line 270, in process
    func(*args, **kwargs)
  File "/home/lfa-01/anaconda3/envs/test_neo/lib/python3.10/site-packages/matplotlib/widgets.py", line 1841, in release
    self._release(event)
  File "/home/lfa-01/anaconda3/envs/test_neo/lib/python3.10/site-packages/matplotlib/widgets.py", line 2396, in _release
    self.onselect(self.eventpress, self.eventrelease)
  File "<decorator-gen-196>", line 12, in _onselect
  File "/home/lfa-01/anaconda3/envs/test_neo/lib/python3.10/site-packages/mne/time_frequency/tfr.py", line 1902, in _onselect
    plot_tfr_topomap(self, ch_type=ch_type, tmin=tmin, tmax=tmax,
TypeError: plot_tfr_topomap() got an unexpected keyword argument 'title'


In [39]:
print(tfr.shape)
print(tfr.mean(axis=(0,1)).shape)

tfr_data_avg = tfr.mean(axis=(0,1))

tfr_data_avg_reshape = tfr_data_avg.reshape([tfr_data_avg.shape[0]*tfr_data_avg.shape[1],1])
print(tfr_data_avg.T.shape)

n=2**7 # frecuencia de downsampling del registro en el dominio temporal
NEW_SFREQ = 512
NEW_TRF_SFREQ = 32

if np.ndim(tfr_data_avg) == 2:
    #step = int(np.floor(tfr_data_avg.shape[1] / n))
    step = int(np.floor(NEW_SFREQ/NEW_TRF_SFREQ))
    print(step)
    print(tfr_data_avg.shape)
    tfr_ds = tfr_data_avg[:, np.arange(1, tfr_data_avg.shape[1], step)]

##
# SpecParam hyperparameters
N_JOBS = -1 # number of jobs for parallel processing
SPEC_PARAM_SETTINGS = {
    'peak_width_limits' :   [4, 20], # default: (0.5, 12.0)) - recommends at least frequency resolution * 2
    'min_peak_height'   :   0.1, 
    'max_n_peaks'       :   4, # (default: inf)
    'peak_threshold'    :   2.0} # (default: 2.0)

for ap_mode in ['fixed']:#['fixed', 'knee']:
    fg = FOOOFGroup(**SPEC_PARAM_SETTINGS, aperiodic_mode=ap_mode, verbose=False)
    fg.fit(freq, tfr_ds.T, n_jobs=N_JOBS)




(15, 1, 128, 10241)
(128, 10241)
(10241, 128)
16
(128, 10241)


In [40]:
group_aperiodic_params = fg.get_params('aperiodic_params') 
print(group_aperiodic_params)

fig1, ax1 = plt.subplots(2,1) #sharex=True,sharey=True
ax1[0].scatter(np.arange(group_aperiodic_params[:,0].shape[0]),group_aperiodic_params[:,0])
ax1[1].scatter(np.arange(group_aperiodic_params[:,1].shape[0]),group_aperiodic_params[:,1])
plt.show()

[[-5.14835086  1.543429  ]
 [-5.12342284  1.52975959]
 [-5.13772081  1.49804576]
 ...
 [-5.23699594  1.46322255]
 [-5.16292545  1.48460238]
 [-5.13898891  1.51582429]]


In [41]:
group_peaks_params = fg.get_params('peak_params') 
print(group_peaks_params)

fig2, ax2 = plt.subplots(1,2)
ax2[0].scatter(group_peaks_params[group_peaks_params[:,0]<8,3],group_peaks_params[group_peaks_params[:,0]<8,0])
ax2[1].scatter(group_peaks_params[group_peaks_params[:,0]<8,3],group_peaks_params[group_peaks_params[:,0]<8,1])
#ax2[1].scatter(group_peaks_params[:,3],group_peaks_params[:,1],c=group_peaks_params[:,0])
plt.show()

[[4.71301421e+00 8.61805774e-01 4.00000000e+00 0.00000000e+00]
 [1.07656166e+01 6.97476717e-01 5.18036623e+00 0.00000000e+00]
 [1.59827120e+01 5.88464136e-01 5.51870411e+00 0.00000000e+00]
 ...
 [1.73364962e+01 3.22215436e-01 7.41802196e+00 6.38000000e+02]
 [5.05385415e+00 9.47143648e-01 4.21946374e+00 6.39000000e+02]
 [1.18806118e+01 3.42521206e-01 9.08974213e+00 6.39000000e+02]]


/home/lfa-01/anaconda3/envs/test_neo/lib/python3.10/site-packages/fooof/objs/group.py:378: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  out = np.array([np.insert(getattr(data, name), 3, index, axis=1)


In [51]:
print(tfr_ds.T.shape)
data = tfr_ds.T

fig3, ax3 = plt.subplots(2,1)
for bin_indx in range(tfr_ds.T.shape[0]):
    
    fm = fg.get_fooof(bin_indx, regenerate=True)

    ax3[0].plot(freq,np.log10(data[bin_indx,:]), alpha = 0.2)
    ax3[1].plot(fm.freqs,fm.fooofed_spectrum_, alpha = 0.2)

plt.show()

(640, 128)
